# Langchain Integrations of Vector Stores


## Chroma

[Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

Chroma runs in various modes. See below for examples of each integrated with LangChain.

- in-memory - in a python script or jupyter notebook
- in-memory with persistance - in a script or notebook and save/load to disk
- in a docker container - as a server running your local machine or in the cloud

Like any other database, you can:

- `.add`
- `.get`
- `.update`
- `.upsert`
- `.delete`
- `.peek`
and `.query` runs the similarity search.

View full docs at [docs](https://docs.trychroma.com/reference/Collection). To access these methods directly, you can do `._collection.method()`



In [ ]:
pip install chromadb

In [ ]:
pip install langchain

In [ ]:
pip install sentence-transformers

In [4]:
# import
from langchain.document_loaders import TextLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

### Basic Example

In this basic example, we take the most recent State of the Union Address, split it into chunks, embed it using an open-source embedding model, load it into Chroma, and then query it.

In [12]:
# create a fake txt file
with open('some_file.txt', 'w') as f:
  f.write(
      """
        My mother's name is Jenny. \n

        My uncle's name is Tim. \n

        We live in Avatar City, Dumbo Street, United States of Dickens.
      """
  )

In [13]:
# load the document and split it into chunks
loader = TextLoader("some_file.txt")
documents = loader.load()

In [14]:
# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [15]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [16]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

In [19]:
# query it
query = "What the mother's name?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

My mother's name is Jenny. My uncle's name is Tim. We live in Avatar City, Dumbo Street, United States of Dickens.


In [20]:
# query it
query = "Where do we live?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

My mother's name is Jenny. My uncle's name is Tim. We live in Avatar City, Dumbo Street, United States of Dickens.


In [21]:
# query it
query = "What is 1+1?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

My mother's name is Jenny. My uncle's name is Tim. We live in Avatar City, Dumbo Street, United States of Dickens.


For more about langchain integrations of vector stores, please see [here](https://python.langchain.com/docs/integrations/vectorstores/chroma).